##### CS 583 Bonus Question 1

I will use Tensorflow implemented Kera's RNN implementation with a custom RNN cell. The Keras sequantial API allows us a custom RNN Cell with keras.layer.RNN, to get a custom RNN layer. 

In [37]:
# Setup and import libraries
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, activations, models
np.random.seed(42)
tf.random.set_seed(42)
from sklearn.model_selection import train_test_split
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

Lets create our datasets

In [38]:
max_bits = 8
n_samples = 100000
samples = np.random.randint(np.power(2, max_bits-1), size=(n_samples, 2))
summed_samples = np.sum(samples, axis=1)
samples_binary_repr = [[np.binary_repr(a, width=max_bits), np.binary_repr(b, width=max_bits)] for a,b in samples]
summed_binary_repr = [np.binary_repr(c, width=max_bits) for c in summed_samples]
x_str = np.array([[list(a), list(b)] for a, b in samples_binary_repr])
y_str = np.array([list(c) for c in summed_binary_repr])
x_flipped = np.flip(x_str, axis=-1)
y_flipped = np.flip(y_str, axis=-1)
x = np.transpose((x_flipped == '1')*1, axes=(0, 2, 1))
y = (y_flipped == '1')*1
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, shuffle=True)

Create the custom RNN cell

In [39]:
class Addcell(layers.Layer):
    def __init__(self, hidden_units, **kwargs):
        super(Addcell, self).__init__(**kwargs)
        self.units = 1
        self.state_size = 1
        self.hidden_units = hidden_units

    def build(self, input_shape):
        self.hidden_kernel = self.add_weight(shape=(input_shape[-1] + self.state_size, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_kernel')
        self.hidden_bias = self.add_weight(shape=(1, self.hidden_units),
                                      initializer='uniform',
                                      name='hidden_bias')
        self.output_kernel = self.add_weight(shape=(self.hidden_units, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_kernel')
        self.output_bias = self.add_weight(shape=(1, self.units + self.state_size),
                                      initializer='uniform',
                                      name='output_bias')
        self.built = True

    def call(self, inputs, states):
        x = tf.concat([inputs, states[0]], axis=-1)
        h = tf.keras.activations.tanh(tf.matmul(x, self.hidden_kernel) + self.hidden_bias)
        o_s = tf.keras.activations.sigmoid(tf.matmul(h, self.output_kernel) + self.output_bias)
        output = o_s[:, :self.units]
        state = o_s[:, self.units:]
        return output, [state]

Lets create the model

In [40]:
model = tf.keras.Sequential(name='Add_Cell')
model.add(layers.RNN(Addcell(3), return_sequences=True, input_shape=(None, 2)))
model.summary()

Model: "Add_Cell"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rnn_5 (RNN)                  (None, None, 1)           20        
Total params: 20
Trainable params: 20
Non-trainable params: 0
_________________________________________________________________


Lets train the model

In [41]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_train, y_train, batch_size=32, epochs=60)
scores = model.evaluate(x_test, y_test, verbose=2)

Epoch 1/60
2813/2813 [==============================] - 5s 2ms/step - loss: 0.6925 - accuracy: 0.4990
Epoch 2/60
2813/2813 [==============================] - 4s 1ms/step - loss: 0.6409 - accuracy: 0.5897
Epoch 3/60
2813/2813 [==============================] - 4s 1ms/step - loss: 0.5037 - accuracy: 0.7346
Epoch 4/60
2813/2813 [==============================] - 4s 1ms/step - loss: 0.4465 - accuracy: 0.7459
Epoch 5/60
2813/2813 [==============================] - 4s 1ms/step - loss: 0.4273 - accuracy: 0.7514
Epoch 6/60
2813/2813 [==============================] - 5s 2ms/step - loss: 0.4227 - accuracy: 0.7514
Epoch 7/60
2813/2813 [==============================] - 4s 2ms/step - loss: 0.4204 - accuracy: 0.7514
Epoch 8/60
2813/2813 [==============================] - 5s 2ms/step - loss: 0.4192 - accuracy: 0.7514
Epoch 9/60
2813/2813 [==============================] - 4s 2ms/step - loss: 0.4184 - accuracy: 0.7514
Epoch 10/60
2813/2813 [==============================] - 5s 2ms/step - loss: 0.417

Not too bad. Accuracy of 92%. Lets use our model to predict a sum. Adam optimizer seems to perform the best. 

In [42]:
#Lets see how we do with an example
max_bits = 8
a = np.random.randint(np.power(2, max_bits-1))
b = np.random.randint(np.power(2, max_bits-1))
a_bin = np.float32(1) * (np.flip(list(np.binary_repr(a, width=max_bits)), axis=-1) == '1')
b_bin = np.float32(1) * (np.flip(list(np.binary_repr(b, width=max_bits)), axis=-1) == '1')
print('a: {}, b: {}'.format(a, b))
print('In binary = a: {}, b: {}'.format(a_bin, b_bin))
a_b = np.stack((a_bin, b_bin), axis=-1).reshape(1,-1,2)
predictions = model(a_b).numpy().flatten()
summed_bin = 1 * (predictions > 0.5)
summed = np.packbits(np.flip(summed_bin , axis=-1))[0]
print('summed: {}'.format(summed))
print('In binary = summed: {}'.format(summed_bin))

a: 11, b: 20
In binary = a: [1. 1. 0. 1. 0. 0. 0. 0.], b: [0. 0. 1. 0. 1. 0. 0. 0.]
summed: 31
In binary = summed: [1 1 1 1 1 0 0 0]
